[Source](https://docs.opencv.org/4.5.2/d3/db4/tutorial_py_watershed.html)

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [4]:
img = cv.imread('water_coins.jpg')
cv.imshow("Original", img)

img_g = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
cv.imshow("Gray", img_g)

ret, thresh = cv.threshold(img_g, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
cv.imshow("Thresh", thresh)

cv.waitKey(0)
cv.destroyAllWindows()

In [5]:
# Noise removal
kernel = np.ones((3,3), np.uint8)
# cv.imshow("Kernel", kernel)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations=2)
# cv.imshow("Opening", opening)

# sure background area
sure_bg = cv.dilate(opening, kernel, iterations=3)

# Finding sure foreground area
dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(), 255,0)

# Finding unknow region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg, sure_fg)

cv.imshow("Unknown", unknown)

ret, thresh = cv.threshold(unknown, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
cv.imshow("Thresh", thresh)


cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
# Marker Labelling
ret, markers = cv.connectedComponents(sure_fg)
# cv.imshow("Makers", markers)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now mark the region of unknown with zero
markers[unknown==255]=0

markers = cv.watershed(img, markers)
img[markers == -1] = [255,0,0]

cv.imshow("Result", img)

cv.waitKey(0)
cv.destroyAllWindows()

error: OpenCV(4.5.3) /tmp/pip-req-build-xw6jtoah/opencv/modules/highgui/src/precomp.hpp:155: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'


## The problem with basic thresholding and contour extraction
[Source](https://www.pyimagesearch.com/2015/11/02/watershed-opencv/)

In [ ]:
# import the necessary packages
from __future__ import print_function
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import argparse
import imutils
import cv2

In [ ]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="path to input image")
args = vars(ap.parse_args())